In [1]:
from gin_train.trainers import KerasTrainer
from kipoi_cadd.data import CaddDataset
from kipoi_cadd.data import cadd_train_valid_data, cadd_serialize_string_row
from kipoi_cadd.data import save_variant_ids, load_variant_ids
from kipoi_cadd.data import train_test_split_indexes
from kipoi_cadd.config import get_data_dir
import pandas as pd
import numpy as np
from random import shuffle
from kipoi_cadd.models import logistic_regression_scikit, logistic_regression_keras
from gin_train.metrics import ClassificationMetrics
from sklearn.model_selection import train_test_split
import lmdb
%load_ext line_profiler

Using TensorFlow backend.


In [2]:
# Create train, test and validation indexes
all_variant_id_file = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/variant_ids.pkl"
training_data_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
train_idx_path = training_data_dir + "train_idx.pkl"
test_idx_path = training_data_dir + "test_idx.pkl"
valid_idx_path = training_data_dir + "valid_idx.pkl"
lmdb_dir = training_data_dir + "lmdb"

In [3]:
variant_ids = load_variant_ids(all_variant_id_file)
variant_ids = variant_ids.values

lmdb_cadd = lmdb.Environment(lmdb_dir, readonly=True)
txn = lmdb_cadd.begin()
lmdb_cadd2 = lmdb.Environment(lmdb_dir, readonly=True)
txn2 = lmdb_cadd.begin()

def myfunc(item):
    var = variant_ids[item]
    buf = bytes(txn.get(var.encode('ascii')))
    
def myfunc2(item):
    var2 = variant_ids[item]
    buf = bytes(txn2.get(var2.encode('ascii')))

In [4]:
%lprun -f myfunc for var in range(310): myfunc(var)

Timer unit: 1e-06 s

Total time: 0.066929 s
File: <ipython-input-3-37c8de56bdcc>
Function: myfunc at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def myfunc(item):
    10       310        403.0      1.3      0.6      var = variant_ids[item]
    11       310      66526.0    214.6     99.4      buf = bytes(txn.get(var.encode('ascii')))

In [5]:
%lprun -f myfunc2 for var in range(310,350): myfunc(var)

Timer unit: 1e-06 s

Total time: 0 s
File: <ipython-input-3-37c8de56bdcc>
Function: myfunc2 at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           def myfunc2(item):
    14                                               var2 = variant_ids[item]
    15                                               buf = bytes(txn2.get(var2.encode('ascii')))

In [9]:
# Debug with other test database
data_dir = "/s/project/kipoi-cadd/data/"
sample_idx = training_data_dir + "sample_variant_ids.pkl"
s_train_idx, s_valid_idx = train_test_split_indexes(sample_idx, test_size=0.4)
save_variant_ids(training_data_dir + "s_train.pkl", s_train_idx)
save_variant_ids(training_data_dir + "s_valid.pkl", s_valid_idx)

s_train, s_valid=cadd_train_valid_data(data_dir + "tests/lmdb_3", training_data_dir + "s_train.pkl", training_data_dir + "s_valid.pkl")

In [10]:
s_it = s_train.batch_train_iter(batch_size=512, num_workers=10, shuffle=True)

In [11]:
next(s_it)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 

In [20]:
from joblib import Parallel, delayed
import pyarrow as pa
from math import sqrt
Parallel(n_jobs=10)(delayed(sqrt)(i**2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [34]:
def get_lmdb_item():
    db_env = lmdb.Environment(lmdb_dir, readonly=True)
    t = db_env.begin()
    var = variant_ids[10]
    buf = bytes(t.get(var.encode('ascii')))
    des = pa.deserialize(buf)
    print(des['metadata']['variant_id'])

In [ ]:
Parallel(n_jobs=10)(delayed(get_lmdb_item)() for i in range(10))
# get_lmdb_item(1)